In [ ]:
!pip install -q transformers datasets

#### Example from https://huggingface.co/tasks/zero-shot-classification

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli")
sequence_to_classify = "Angela Merkel is a politician in Germany and leader of the CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

### Deployment as an endpoint on SageMaker

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

In [ ]:
role = sagemaker.get_execution_role()

In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli',
    'HF_TASK':'zero-shot-classification'
}
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, 
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type="ml.g4dn.xlarge" # ec2 instance type
)


In [ ]:
text = {
    "inputs": "one day I will see the world", 
    "parameters": {
        "candidate_labels": ['travel', 'cooking', 'dancing']
    }
}

In [ ]:
predictor.predict(text)

In [ ]:
# predictor.delete_endpoint()

## Inference with SDK Boto3

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("./dataset/sem_eval_2018_task_1_test.csv")

In [ ]:
label_name = [label for label in df.columns if label not in ['ID', 'Tweet']]
label_name

In [ ]:
label_index = df.iloc[:,2:].astype(int)
label_index.head(5)

In [ ]:
text_tweet = df.loc[:,["Tweet"]].values.tolist()

In [ ]:
text_tweet[0]

In [ ]:
import boto3
import json

In [ ]:
runtime = boto3.client("sagemaker-runtime")

In [ ]:
endpoint_name = "huggingface-pytorch-inference-2023-03-16-07-08-39-697"

In [ ]:
input_json = {
    "inputs": text_tweet[0], 
    "parameters": {
        "candidate_labels": label_name
    }
}

In [ ]:
input_json

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(input_json),
    ContentType="application/json",
)

print(response["Body"].read())